## Todo
- Rearrange cells to run sequentially top2bottom
- remove unnecessary cells
- add markdown to explain code cells

In [20]:
## Not needed in python file
import warnings
warnings.filterwarnings("ignore")

In [21]:
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import requests
import pickle
from collections import defaultdict
from math import log
import math
import time
import re
from tqdm import tqdm_notebook as tqdm
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [165]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\HP 1040
[nltk_data]     G2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\HP 1040
[nltk_data]     G2\AppData\Roaming\nltk_data...


True

In [240]:
#search_term = input("What product do you want to search for? ")

url = f"https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/?ordering=publicationYearThenTitle&descending=true"
#url = f"https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/"
page = requests.get(url, verify=False).text
doc = BeautifulSoup(page, "html.parser")

#print(doc)

In [241]:
num_pub = int(doc.find("li", class_="search-pager-information").text.strip().strip("/n").split()[-2])
num_pub

220

In [242]:
q, r = divmod(num_pub, 50)
combined_pattern = re.compile(r"(^[A-Z][a-z]*(?:\s[A-Z][a-z]*)*,\s[A-Z][a-z]*(?:\s[A-Z][a-z]*)*(?:,\s[A-Z][a-z]*(?:\s[A-Z][a-z]*)*)*)|(^[A-Z][a-z]*(?:-[A-Z][a-z]*)*,\s[A-Z][a-z]*(?:-[A-Z][a-z]*)*(?:,\s[A-Z][a-z]*(?:-[A-Z][a-z]*)*)*)")
combined_pattern = re.compile(r"(^\w+(?:\s\w+)*,\s\w+(?:\s\w+)*(?:,\s\w+(?:\s\w+)*)*)|(^\w+(?:-\w+)*,\s\w+(?:-\w+)*(?:,\s\w+(?:-\w+)*)*)")

In [243]:
def to_str(string):
    return str(unicodedata.normalize('NFKD', string).encode('ascii', 'ignore'), "utf-8")

In [244]:
def get_authors(list_result):
    authors = []
    for i in list_result.find_all("span"):
        if combined_pattern.search(to_str(i.text)):
            authors.append(i.text)
    return authors

def get_author_link(name_links, authors):
    author_link = [None for i in range(len(authors))]
    for i in name_links:
        index = authors.index(i.text)
        author_link[index] = i.get("href")
    return author_link

In [245]:
def get_each_pub(list_result):
    title = list_result.find("h3", class_="title").find("a").text
    title_link = list_result.find("h3", class_="title").find("a").get("href")
    yr = int(list_result.find("span", class_="date").text.split()[-1])
    authors = get_authors(list_result)
    name_links = list_result.find_all("a", rel="Person")
    authors_link = get_author_link(name_links, authors)
    return (authors, authors_link, yr, title, title_link)

In [246]:
def get_pub(doc, num):
    for i in tqdm(range(num)):
        list_result = doc.find(class_="list-results").find(class_=f"list-result-item list-result-item-{i}")

        authors, authors_link, yr, title, title_link = get_each_pub(list_result)
        publications["Authors"].append(authors)
        publications["Authors_link"].append(authors_link)
        publications["Publication_Year"].append(yr)
        publications["Title"].append(title)
        publications["link"].append(title_link)

In [247]:
publications = {"Authors":[], "Authors_link":[], "Publication_Year":[], "Title":[], "link":[]}
for i in tqdm(range(q+1)):
    if i == 0:
        url = f"https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/?ordering=publicationYearThenTitle&descending=true"
    else:
        url = f"https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/?ordering=publicationYearThenTitle&descending=true&page={1}"
    
    
    page = requests.get(url, verify=False).text
    doc = BeautifulSoup(page, "html.parser")
    if i != 4:
        get_pub(doc, 50)
    else:
        get_pub(doc, r)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [240]:
publications

{'Authors': [['Pay, M. L.',
   'Christensen, J.',
   'He, F.',
   'Roden, L.',
   'Ahmed, H.',
   'Foo, M.'],
  ['Fanous, M.', 'Eden, J.', 'Remesan, R.', 'Daneshkhah, A.'],
  ['Tariq, M.', 'Palade, V.', 'Ma, Y.', 'Altahhan, A.'],
  ['You, Q.', 'Li, C.', 'Sun, J.', 'Palade, V.', 'Pan, F.'],
  ['Jiang, X.', 'Liu, J.'],
  ['Ganeshamoorthy, S.', 'Roden, L.', 'Klepl, D.', 'He, F.'],
  ['Gunawardena, S. R. A. S.',
   'Sarrigiannis, P. G.',
   'Blackburn, D. J.',
   'He, F.'],
  ['Shah, H.', 'Roberts, F.'],
  ['Zhao, R.', 'Dai, X.', 'Hu, P.', 'Munteanu, A.'],
  ['Röst, G.', 'Sadeghimanesh, A.'],
  ['Charroud, A.', 'El Moutaouakil, K. E.', 'Palade, V.', 'Yahyaouy, A.'],
  ['Verba, N.', 'Nixon, J. D.', 'Gaura, E.', 'Dias, L. A.', 'Halford, A.'],
  ['Magaraja, A. D.',
   'Rajapackiyam, E.',
   'Kanagaraj, V.',
   'Kanagaraj, S. J.',
   'Kotecha, K.',
   'Vairavasundaram, S.',
   'Mehta, M.',
   'Palade, V.'],
  ['Ajao-Olarinoye, M.'],
  ['Abu-Monshar, A.', 'Al Bazi, A.', 'Palade, V.'],
  ['Li, C

In [248]:
df = pd.DataFrame(publications)

In [249]:
df.sample(10)

,Authors,Authors_link,Publication_Year,Title,link
151,"[Deshpande, S., Shuttleworth, J., Yang, J., Ta...","[None, https://pureportal.coventry.ac.uk/en/pe...",2022,LIFT: lncRNA identification and function-predi...,https://pureportal.coventry.ac.uk/en/publicati...
148,"[Sharifzadeh, S., Amiri, S., Abdi Jalebi, S.]",[https://pureportal.coventry.ac.uk/en/persons/...,2021,A New Method for Semi-Supervised Segmentation ...,https://pureportal.coventry.ac.uk/en/publicati...
129,"[Cheruvallil-Contractor, S., Halford, A., Boti...",[https://pureportal.coventry.ac.uk/en/persons/...,2022,"The politics of Matching: Ethnicity, Religion ...",https://pureportal.coventry.ac.uk/en/publicati...
42,"[Grawemeyer, B., Halloran, J., England, M., Cr...",[https://pureportal.coventry.ac.uk/en/persons/...,2022,Feedback and Engagement on an Introductory Pro...,https://pureportal.coventry.ac.uk/en/publicati...
93,"[Aflaki, A., Gitizadeh, M., Razavi-Far, R., Pa...","[None, None, None, https://pureportal.coventry...",2021,A hybrid framework for detecting and eliminati...,https://pureportal.coventry.ac.uk/en/publicati...
170,"[Charroud, A., El Moutaouakil, K., Yahyaouy, A...","[None, None, None, None, https://pureportal.co...",2022,Rapid Localization and Mapping Method Based on...,https://pureportal.coventry.ac.uk/en/publicati...
191,"[Dias, L. A., Damasceno, A. M. P., Gaura, E., ...","[None, None, https://pureportal.coventry.ac.uk...",2021,A full-parallel implementation of Self-Organiz...,https://pureportal.coventry.ac.uk/en/publicati...
36,"[Klepl, D., He, F., Wu, M., Blackburn, D. J., ...",[https://pureportal.coventry.ac.uk/en/persons/...,2022,EEG-based Graph Neural Network Classification ...,https://pureportal.coventry.ac.uk/en/publicati...
1,"[Fanous, M., Eden, J., Remesan, R., Daneshkhah...",[https://pureportal.coventry.ac.uk/en/persons/...,2023,Challenges and prospects of climate change imp...,https://pureportal.coventry.ac.uk/en/publicati...
132,"[Jiang, X.]",[https://pureportal.coventry.ac.uk/en/persons/...,2022,Trustworthiness of systematic review automatio...,https://pureportal.coventry.ac.uk/en/publicati...


In [250]:
df["Authors"][0], df["Authors_link"][0]

(['Pay, M. L.',
  'Christensen, J.',
  'He, F.',
  'Roden, L.',
  'Ahmed, H.',
  'Foo, M.'],
 ['https://pureportal.coventry.ac.uk/en/persons/miao-lin-pay',
  'https://pureportal.coventry.ac.uk/en/persons/jesper-christensen',
  'https://pureportal.coventry.ac.uk/en/persons/fei-he',
  'https://pureportal.coventry.ac.uk/en/persons/laura-roden',
  None,
  None])

In [251]:
df.shape

(220, 5)

In [252]:
#df.to_csv("papers.csv", index=False)

In [3]:
# Connect to database (creates the database if it doesn't exist)
conn = sqlite3.connect('publications.db')

In [284]:
# Create a cursor object to execute SQL commands
c = conn.cursor()

# Create Authors table
c.execute('''CREATE TABLE Authors (
                id INTEGER PRIMARY KEY,
                Name TEXT,
                Link TEXT,
                Paper INTEGER,
                FOREIGN KEY (Paper) REFERENCES Papers(id)
            )''')

# Create Papers table
c.execute('''CREATE TABLE Papers (
                id INTEGER PRIMARY KEY,
                Title TEXT,
                Link TEXT,
                PublicationYear INTEGER
            )''')

# Commit changes and close connection
conn.commit()
conn.close()

In [22]:
# Connect to database (creates the database if it doesn't exist)
conn = sqlite3.connect('publications.db')
# Create a cursor object to execute SQL commands
c = conn.cursor()

In [48]:
#conn.close()

In [285]:


# Insert data into Papers table
for index, row in df.iterrows():
    c.execute('''INSERT INTO Papers (Title, Link, PublicationYear) VALUES (?, ?, ?)''', (row['Title'], row['link'], row['Publication_Year']))


# Commit changes and close connection
conn.commit()
conn.close()

In [304]:
# Insert data into Authors table
for index, row in df.iterrows():
    for i in zip(row['Authors'], row['Authors_link']):
        c.execute('''INSERT INTO Authors (Name, Link, Paper) VALUES (?, ?, ?)''', (i[0], i[1], index+1))
# Commit changes and close connection
conn.commit()

In [288]:
try:

    # connect to a databse
    conn = sqlite3.connect('publications.db')

    # If sqlite3 makes a connection with python
    # program then it will print "Connected to SQLite"
    # Otherwise it will show errors
    print("Connected to SQLite")

    # Getting all tables from sqlite_master
    sql_query = """SELECT name FROM sqlite_master
    WHERE type='table';"""

    # Creating cursor object using connection object
    cursor = conn.cursor()

    # executing our sql query
    cursor.execute(sql_query)
    print("List of tables\n")

    # printing all tables list
    print(cursor.fetchall())

except sqlite3.Error as error:
    print("Failed to execute the above query", error)

finally:

    # Inside Finally Block, If connection is
    # open, we need to close it
    if conn:

        # using close() method, we will close
        # the connection
        conn.close()

        # After closing connection object, we
        # will print "the sqlite connection is
        # closed"
        print("the sqlite connection is closed")


Connected to SQLite
List of tables

[('Authors',), ('Papers',)]
the sqlite connection is closed


In [5]:
papers = pd.read_sql_query("SELECT * FROM Papers", conn)
papers.head()

,id,Title,Link,PublicationYear
0,1,An Extended Plant Circadian Clock Model for Ch...,https://pureportal.coventry.ac.uk/en/publicati...,2023
1,2,Challenges and prospects of climate change imp...,https://pureportal.coventry.ac.uk/en/publicati...,2023
2,3,Diabetic Retinopathy Detection Using Transfer ...,https://pureportal.coventry.ac.uk/en/publicati...,2023
3,4,Entropy‐based lamarckian quantum‐behaved parti...,https://pureportal.coventry.ac.uk/en/publicati...,2023
4,5,Extracting the Evolutionary Backbone of Scient...,https://pureportal.coventry.ac.uk/en/publicati...,2023


In [6]:
papers.shape

(220, 4)

In [7]:
papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               220 non-null    int64 
 1   Title            220 non-null    object
 2   Link             220 non-null    object
 3   PublicationYear  220 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 7.0+ KB


In [23]:
author = pd.read_sql_query("SELECT * FROM Authors", conn)
author.head()

,id,Name,Link,Paper
0,1,"Pay, M. L.",https://pureportal.coventry.ac.uk/en/persons/m...,1
1,2,"Christensen, J.",https://pureportal.coventry.ac.uk/en/persons/j...,1
2,3,"He, F.",https://pureportal.coventry.ac.uk/en/persons/f...,1
3,4,"Roden, L.",https://pureportal.coventry.ac.uk/en/persons/l...,1
4,5,"Ahmed, H.",None,1


In [24]:
author.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 819 entries, 0 to 818
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      819 non-null    int64 
 1   Name    819 non-null    object
 2   Link    349 non-null    object
 3   Paper   819 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 25.7+ KB


In [31]:
df_a = author[~author["Link"].isna()].drop_duplicates(subset='Name', keep="last")
df_a.shape

(50, 4)

In [37]:
author[~author["Link"].isna()].groupby("Name")["Paper"].count()

Name
Ahmed, M. L.                   4
Ajao-Olarinoye, M.             1
Al Bazi, A.                    7
Ashraf, R. J.                  3
Azizsafaei, M.                 1
Babatunde, A.                  1
Barket, R.                     5
Benkhelifa, F.                 8
Brusey, J.                    12
Chatrabgoun, O.               12
Cheruvallil-Contractor, S.     7
Christensen, J.                2
Daneshkhah, A.                23
Deo, A.                        4
Donnelly, J.                   1
Eden, J.                       1
England, M.                   33
Fanous, M.                     4
Ganeshamoorthy, S.             1
Gaura, E.                     18
Grawemeyer, B.                 1
Gunawardena, S. R. A. S.       1
Halford, A.                   20
Halloran, J.                   1
He, F.                        12
Horri, N.                      3
Hu, P.                         1
Jess, B. J.                    1
Jiang, X.                      9
Kasap, S.                      6
Kim, 

In [ ]:
# Set up the database connection
conn = sqlite3.connect('publications.db')
c = conn.cursor()

In [4]:
# Define the pre-processing functions
stop_words = set(stopwords.words('english'))
#stemmer = PorterStemmer()
lemma = nltk.wordnet.WordNetLemmatizer()

In [187]:
#stop_words

In [5]:
def preprocess(text):
#     # Tokenize the text
#     tokens = text.split()
    # Remove non-alphanumeric characters and split into tokens
    tokens = re.findall(r'\w+', text)
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    
    # Convert the tokens to lowercase
    tokens = [token.lower() for token in tokens]
    
    # Lematize the tokens
    tokens = [lemma.lemmatize(token) for token in tokens]
    
    # Remove punctuation and special characters
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
    
    # Remove single-letter tokens
    tokens = [token for token in tokens if len(token) > 1]
    
    return tokens

In [218]:
def create_inverted_index(table_name):
    inverted_index = {}
    if table_name == "Papers":
        c.execute("SELECT rowid, Title, PublicationYear FROM Papers")
        papers = c.fetchall()
        num_papers = len(papers)
        for paper in papers:
            paper_id = paper[0]
            title = paper[1]
            year = paper[2]
            text = title + " " + str(year)
            tokens = preprocess(text)
            for token in tokens:
                if token not in inverted_index:
                    inverted_index[token] = {}
                if paper_id not in inverted_index[token]:
                    inverted_index[token][paper_id] = 0
                inverted_index[token][paper_id] += 1
        for token in inverted_index.keys():
            df = len(inverted_index[token])
            idf = math.log(num_papers / df)
            for paper_id in inverted_index[token].keys():
                tf = inverted_index[token][paper_id]
                tfidf = tf * idf
                inverted_index[token][paper_id] = tfidf
    elif table_name == "Authors":
        c.execute("SELECT Authors.Name, Papers.rowid FROM Authors JOIN Papers ON Authors.Paper=Papers.rowid")
        author_paper_rows = c.fetchall()
        num_papers = c.execute("SELECT COUNT(DISTINCT Papers.rowid) FROM Papers").fetchone()[0]
        for row in author_paper_rows:
            author_name = row[0]
            paper_id = row[1]
            text = author_name
            tokens = preprocess(text)
            for token in tokens:
                if token not in inverted_index:
                    inverted_index[token] = {}
                if paper_id not in inverted_index[token]:
                    inverted_index[token][paper_id] = 0
                inverted_index[token][paper_id] += 1
        for token in inverted_index.keys():
            df = len(inverted_index[token])
            idf = math.log(num_papers / df)
            for paper_id in inverted_index[token].keys():
                tf = inverted_index[token][paper_id]
                tfidf = tf * idf
                inverted_index[token][paper_id] = tfidf
    else:
        print("Error: Invalid table name.")
    
    return inverted_index


In [219]:
inverted_index_paper = create_inverted_index("Papers")
inverted_index_author = create_inverted_index("Authors")

In [220]:
inverted_index_combined = {}
for token in inverted_index_paper.keys():
    if token not in inverted_index_combined:
        inverted_index_combined[token] = inverted_index_paper[token]
    else:
        inverted_index_combined[token].update(inverted_index_paper[token])
        
for token in inverted_index_author.keys():
    if token not in inverted_index_combined:
        inverted_index_combined[token] = inverted_index_author[token]
    else:
        inverted_index_combined[token].update(inverted_index_author[token])


In [221]:
inverted_index_combined

{'an': {1: 2.5604142022961454,
  15: 2.5604142022961454,
  16: 2.5604142022961454,
  17: 2.5604142022961454,
  18: 2.5604142022961454,
  19: 2.5604142022961454,
  20: 2.5604142022961454,
  37: 2.5604142022961454,
  83: 2.5604142022961454,
  97: 2.5604142022961454,
  100: 2.5604142022961454,
  133: 2.5604142022961454,
  147: 2.5604142022961454,
  150: 2.5604142022961454,
  183: 2.5604142022961454,
  197: 2.5604142022961454,
  200: 2.5604142022961454},
 'extended': {1: 5.393627546352362},
 'plant': {1: 4.700480365792417, 33: 4.700480365792417},
 'circadian': {1: 5.393627546352362},
 'clock': {1: 5.393627546352362},
 'model': {1: 2.908720896564361,
  2: 2.908720896564361,
  39: 2.908720896564361,
  41: 2.908720896564361,
  46: 2.908720896564361,
  48: 2.908720896564361,
  88: 2.908720896564361,
  100: 2.908720896564361,
  138: 2.908720896564361,
  150: 2.908720896564361,
  188: 2.908720896564361,
  200: 2.908720896564361},
 'characterising': {1: 4.700480365792417, 27: 4.700480365792417},


In [6]:
#Load Initial inverted Index
with open('inverted_index.pkl', 'rb') as fp:
    inverted_index_combined = pickle.load(fp)

In [12]:
def query_inverted_index(query, inverted_index):
    if query == "":
        print("You need to enter something to search")
        return
    conn = sqlite3.connect('publications.db')
    c = conn.cursor()
    tokens = preprocess(query)
    scores = {}
    for token in tokens:
        if token in inverted_index:
            for paper_id, score in inverted_index[token].items():
                if paper_id not in scores:
                    scores[paper_id] = 0
                scores[paper_id] += score
    
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    results = []
    for paper_id, score in sorted_scores:
        c.execute("""SELECT DISTINCT Papers.Title, Papers.Link, Papers.PublicationYear, Authors.Name, Authors.Link
                     FROM Papers 
                     INNER JOIN Authors ON Papers.Id = Authors.Paper
                     WHERE Papers.Id=?""", (paper_id,))
        row = c.fetchone()
        if row:
            title, link, year, authors, authors_link = row
            results.append((title, link, year, score, authors, authors_link))
        print(f"Title: {title}")
        print(f"Link: {link}")
        print(f"Year: {year}")
        print(f"Author: {authors}")
        print(f"Authors Link: {authors_link}")
        print(f"Relevance Score: {score}")
        print("\n")
    conn.close()
    return #results


In [16]:
q = input("Enter your search: ")
print()
query_inverted_index(q, inverted_index_combined)

Enter your search: Machines

Title: Fast, detailed, accurate simulation of a thermal car-cabin using machine-learning
Link: https://pureportal.coventry.ac.uk/en/publications/fast-detailed-accurate-simulation-of-a-thermal-car-cabin-using-ma
Year: 2022
Author: Fast, detailed, accurate simulation of a thermal car-cabin using machine-learning
Authors Link: None
Relevance Score: 5.3981627015177525


Title: Detection of sleep apnea using Machine learning algorithms based on ECG Signals: A comprehensive systematic review
Link: https://pureportal.coventry.ac.uk/en/publications/detection-of-sleep-apnea-using-machine-learning-algorithms-based-
Year: 2022
Author: Salari, N.
Authors Link: None
Relevance Score: 2.179286876649552


Title: Machine Learning for Computer Algebra
Link: https://pureportal.coventry.ac.uk/en/publications/machine-learning-for-computer-algebra
Year: 2022
Author: Barket, R.
Authors Link: https://pureportal.coventry.ac.uk/en/persons/rashid-barket
Relevance Score: 2.17928687664

In [17]:
q = input("Enter your search: ")
print()
query_inverted_index(q, inverted_index_combined)

Enter your search: Artificial intelligence and machine learnig is the new oil

Title: Artificial Intelligence in Cars: How Close Yet Far Are We from Fully Autonomous Vehicles?
Link: https://pureportal.coventry.ac.uk/en/publications/artificial-intelligence-in-cars-how-close-yet-far-are-we-from-ful
Year: 2022
Author: Palade, V.
Authors Link: https://pureportal.coventry.ac.uk/en/persons/vasile-palade
Relevance Score: 10.10317822247556


Title: Fast, detailed, accurate simulation of a thermal car-cabin using machine-learning
Link: https://pureportal.coventry.ac.uk/en/publications/fast-detailed-accurate-simulation-of-a-thermal-car-cabin-using-ma
Year: 2022
Author: Fast, detailed, accurate simulation of a thermal car-cabin using machine-learning
Authors Link: None
Relevance Score: 5.3981627015177525


Title: An Effective Swarm Intelligence Optimization Algorithm for Flexible Ligand Docking
Link: https://pureportal.coventry.ac.uk/en/publications/an-effective-swarm-intelligence-optimization-al

In [19]:
q = input("Enter your search: ")
print()
query_inverted_index(q, inverted_index_combined)

Enter your search: The basic need of a man

Title: Extracting the Evolutionary Backbone of Scientific Domains: The Semantic Main Path Network Approach Based on Citation Context Analysis
Link: https://pureportal.coventry.ac.uk/en/publications/extracting-the-evolutionary-backbone-of-scientific-domains-the-se
Year: 2023
Author: Jiang, X.
Authors Link: https://pureportal.coventry.ac.uk/en/persons/xiaorui-jiang
Relevance Score: 2.759105371902494


Title: Off the boil? The challenges of monitoring cooking behaviour in refugee settlements
Link: https://pureportal.coventry.ac.uk/en/publications/off-the-boil-the-challenges-of-monitoring-cooking-behaviour-in-re
Year: 2022
Author: Halford, A.
Authors Link: https://pureportal.coventry.ac.uk/en/persons/alison-halford
Relevance Score: 2.759105371902494


Title: The benefits of clustering in cylindrical algebraic decomposition
Link: https://pureportal.coventry.ac.uk/en/publications/the-benefits-of-clustering-in-cylindrical-algebraic-decomposition
Yea